In [1]:
# Проверка возможности использования xgbooster в основном, либо предварительно (2017-06-30)

In [2]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from __future__ import print_function

In [3]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [4]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [5]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [44]:
binsData = 32

trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
if False : # True :
    trDirI = trDirJPG
    teDirI = teDirJPG
    ttExt  = '.jpg'
else :
    trDirI = trDirTIF
    teDirI = teDirTIF
    ttExt  = '.tif'
trWork, teWork = '../Work/Train', '../Work/Test'
print (trDirI,teDirI, trWork, teWork)
print (trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG)

../Data/train-tif-v2 ../Data/test-tif-v2 ../Work/Train ../Work/Test
../Data/train_v2.csv ../Data/train-tif-v2 ../Data/train-jpg ../Data/test-tif-v2 ../Data/test-jpg-v2


In [45]:
xx=os.listdir(trDirI); xx.sort(); xx[:6]

['train_0.tif',
 'train_1.tif',
 'train_10.tif',
 'train_100.tif',
 'train_1000.tif',
 'train_10000.tif']

In [11]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [12]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [13]:
# Add features (0 or 1) for every label as dataframe field 
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
#labels_df.head()

In [14]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [15]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
#labelList[:6,:]
#labels_df.head()

In [16]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

def formImHist (nf, size, printOK=False) :
    
    def hist1 (nf1,bins) :
        h,_ = np.histogram(nf1.ravel(),bins=bins)
        return h.tolist()
    
    def histN (nf,bins) :
        h = []
        for i in range(0,nf.shape[2]) :
            h = h + hist1(nf[:,:,i],bins=bins);
        return h
             
    def histNX (nf,bins) :
        h = []
        for i in range(0,nf.shape[2]) :
            h = h + hist1(nf[:,:,i],bins=bins[i]);
        return h
             
    nx = None
    try : 
        ni = cv.imread(nf,-1); #print(ni.shape)
        if (ni is not None) :
            
            if printOK : print(nf,ni.shape)

            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : 
                print('----- error ---- shape:',ni.shape,nf);
            if  ni.shape[2]==3 :
                bins = [x for x in range(0,255+int(round(256.0/size)),int(round(256.0/size)))]
                nx = histN(ni,bins)
            if  ni.shape[2]==4 : 
                bins0 = [x      for x in range(0,65535+int(round(65535.0/size)),int(round(65535.0/size)))]
                l,h,d = -16000.0,16000.0,1000.0
                bins1 = [float(x/d) for x in range(int(l),int(h+int(round((h-l)/size))), int(round((h-l)/size)))] 
                bins  = [bins0,bins0,bins0,bins0,bins1,bins1]
                r,g,b,n = ni[:,:,2], ni[:,:,1], ni[:,:,0], ni[:,:,3]
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                ni      = np.array([r,g,b,n,dv,dw]).T; 
                nx = histNX(ni,bins)
            
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [17]:
#size= 75
#bins1 =[float(x/1000.0) for x in range(-1*1000,1*1000+int(round(2000.0/size)), int(round(2000.0/size)))] 
#bins1
#75*4,trX.shape,trX[1,75*0:75*6]

In [46]:
trX, trY, i, size = [],[], 0, len(nameList)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    nf = os.path.join(trDirI,nn+ttExt);
    nx = formImHist(nf,binsData,printOK=False)
    if (nx is not None) :
        #rr=np.save(os.path.join(trWork,nn+".npy"),nx);
        #trX.append(nn+".npy")
        trX.append(nx)
        trY.append(True)
    else : 
        trY.append(False)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)

print(datetime.datetime.now())
trX = np.array(trX);
trY = labelList[trY];
print(len(nameList),trX.shape,trY.shape)
#trXY=pd.DataFrame(trY); trXY['name']=trX; trXY.head()
#trXY.to_pickle(os.path.join(trWork,"listFiles.pkl"))

2017-07-02 09:17:55.551197 40479 40479
2017-07-02 09:22:15.023269 	 5000 	 train_4999
2017-07-02 09:26:33.755457 	 10000 	 train_9999
2017-07-02 09:30:48.234418 	 15000 	 train_14999
2017-07-02 09:34:59.477844 	 20000 	 train_19999
2017-07-02 09:39:04.663523 	 25000 	 train_24999
2017-07-02 09:43:27.864650 	 30000 	 train_29999
2017-07-02 09:47:51.811525 	 35000 	 train_34999
2017-07-02 09:52:15.504120 	 40000 	 train_39999
2017-07-02 09:52:39.708127
40479 (40479, 192) (40479, 17)


In [47]:
trX.shape,trY.shape

((40479, 192), (40479, 17))

In [48]:
np.save('../Work/train-data-XX'+str(binsData)+ttExt+'.npy',trX)
np.save('../Work/train-data-YY'+str(binsData)+ttExt+'.npy',trY)

In [49]:
del(trX); del(trY);

In [50]:
trX = np.load('../Work/train-data-XX'+str(binsData)+ttExt+'.npy')
trY = np.load('../Work/train-data-YY'+str(binsData)+ttExt+'.npy')
print(trX.shape,trY.shape)

(40479, 192) (40479, 17)


In [ ]:
# Forming output dataset for predicting --> trOX, trOY
del(trX)
del(trY)

In [ ]:
nameAsk = os.listdir(teDirI); print(len(nameAsk))
trOX, trOY, i, size = [], [], 0, len(nameAsk)
print(datetime.datetime.now(),len(nameAsk),size)
for nn in nameAsk[0:size] :
    nf = os.path.join(teDirI,nn);
    nx = formImHist(nf,binsData) #,printOK=True)
    if (nx is not None) :
        trOX.append(nx)
        trOY.append(nn)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)

print(datetime.datetime.now())

#assert (size!=len(trOY)), "Wrong files {} != {}".format(size,len(trOY))

61191
2017-07-02 09:52:42.014824 61191 61191
2017-07-02 09:56:47.538964 	 5000 	 test_15649.tif
2017-07-02 10:00:45.806645 	 10000 	 test_32423.tif
2017-07-02 10:04:42.808560 	 15000 	 test_36264.tif
2017-07-02 10:08:42.314236 	 20000 	 test_10894.tif
2017-07-02 10:12:41.354721 	 25000 	 file_9502.tif
2017-07-02 10:16:33.920637 	 30000 	 file_14128.tif


In [ ]:
trOX = np.array(trOX);
trOY = np.array([os.path.splitext(x)[0] for x in trOY]);
print(len(nameAsk),trOX.shape,trOY.shape)

In [ ]:
#trOX[20],trOY[20]

In [ ]:
# Saving & Loading

In [ ]:
np.save('../Work/test-data-XX'+str(binsData)+ttExt+'.npy',trOX)
np.save('../Work/test-data-YY'+str(binsData)+ttExt+'.npy',trOY)

In [ ]:
trOX = np.load('../Work/test-data-XX'+str(binsData)+ttExt+'.npy')
trOY = np.load('../Work/test-data-YY'+str(binsData)+ttExt+'.npy')
print(trOX.shape,trOY.shape)